In [1]:
DATA_NAME = 'segmentAnything-full' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = 'blue'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_blue_diagonal_wavelet_full_segmentAnything.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10
2,3449.75,-0.02,-0.00,0.02,0.00,-0.00,0.00,-0.00,-0.00
3,-0.02,867.53,-0.01,0.00,-0.00,0.00,0.00,0.00,0.00
4,-0.00,-0.01,209.91,0.00,-0.00,-0.00,-0.00,-0.00,0.00
5,0.02,0.00,0.00,48.43,0.00,-0.00,0.00,0.00,0.00
6,0.00,-0.00,-0.00,0.00,10.93,-0.00,-0.00,0.00,0.00
7,-0.00,0.00,-0.00,-0.00,-0.00,2.47,0.00,0.00,0.00
8,0.00,0.00,-0.00,0.00,-0.00,0.00,0.59,0.00,0.00
9,-0.00,0.00,-0.00,0.00,0.00,0.00,0.00,0.15,-0.00
10,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,0.04


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10
2,1.00000,-0.00001,-0.00000,0.00004,0.00000,-0.00003,0.00002,-0.00002,-0.00001
3,-0.00001,1.00000,-0.00001,0.00001,-0.00003,0.00002,0.00007,0.00001,0.00001
4,-0.00000,-0.00001,1.00000,0.00004,-0.00000,-0.00000,-0.00005,-0.00001,0.00002
5,0.00004,0.00001,0.00004,1.00000,0.00000,-0.00002,0.00007,0.00001,0.00001
6,0.00000,-0.00003,-0.00000,0.00000,1.00000,-0.00001,-0.00001,0.00003,0.00001
7,-0.00003,0.00002,-0.00000,-0.00002,-0.00001,1.00000,0.00002,0.00001,0.00001
8,0.00002,0.00007,-0.00005,0.00007,-0.00001,0.00002,1.00000,0.00003,0.00001
9,-0.00002,0.00001,-0.00001,0.00001,0.00003,0.00001,0.00003,1.00000,-0.00001
10,-0.00001,0.00001,0.00002,0.00001,0.00001,0.00001,0.00001,-0.00001,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.03966388879547659

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[1.32378339e+06 8.23418319e+04 4.72230284e+03 2.44526287e+02
 1.19576546e+01 5.71807661e-01 2.94245327e-02 1.52234296e-03
 4.45520834e-36]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8
0,0.999409,-0.033489,-0.007557,-0.001732,-0.000391,-0.000089,-0.000021,-0.000006,-0.000001
1,0.033172,0.998734,-0.036917,-0.007972,-0.001797,-0.000404,-0.000095,-0.000025,-0.000007
2,0.008691,0.036275,0.998430,-0.040823,-0.008672,-0.001950,-0.000470,-0.000121,-0.000031
3,0.002323,0.009277,0.040011,0.997956,-0.047774,-0.010156,-0.002417,-0.000624,-0.000164
4,0.000628,0.002502,0.010347,0.046646,0.996975,-0.059642,-0.013434,-0.003442,-0.000906
5,0.000176,0.000698,0.002886,0.012539,0.057656,0.994435,-0.084683,-0.020240,-0.005301
6,0.000055,0.000217,0.000906,0.003909,0.017394,0.080034,0.986799,-0.135828,-0.032463
7,0.000020,0.000079,0.000329,0.001421,0.006302,0.028001,0.120820,0.957412,-0.260651
8,0.000011,0.000044,0.000184,0.000796,0.003532,0.015663,0.065361,0.253951,0.964872


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0005910573656284779,
 0.0012659190928351327,
 0.0015698306974117893,
 0.0020441341735925045,
 0.003024501367734911,
 0.0055650740951440625,
 0.013200763115911784,
 0.0425880935131594,
 0.0351279324066206]